# Proyecto Matemáticas Discretas

A continuación se presenta el proyecto de la asignatura Matemáticas Discretas del IIMAS

## Objetivo: 
Analizar los datos de las materias del ciclo 2021-1 de la Facultad de Ciencias de la UNAM, para crear un programa que acomode el horario de un alumno, dado su avance escolar y sus intereses para materias optativas.

### Nota : 
Este notebook solo corre en **LINUX**, en caso de estar usando un SO distinto, pasar a otros notebooks, ya que este archivo se usa para mostrar el funcionamiento de extracción de datos a un archivo llamado `materias_limpio` que ya se encuentra en la carpeta `ciencias_horarios`

## Extracción de los datos



La obtención de los datos fue mediante el sitio de la facultad https://web.fciencias.unam.mx/docencia/horarios/indiceplan/20211/217, en donde se obtuvo el documento `20211.pdf`

Para ello, vamos a convertir el pdf a texto con la siguiente función:

In [136]:
import subprocess
import re
import json

In [30]:
def normalizar(s):
    '''
    Función para quitar acentos de una cadena de texto
    
    :param str s: Cadena a quitarle acentos
    
    :returns str: Cadena sin acentos
    '''
    reemplazar = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ('á', 'a'), #Existen dos tipos de acentos en la codificación 
        ("é", "e"),
        ("ı́", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in reemplazar:
        s = s.replace(a, b)
    return s


def pdftotext(pdf, page=None):
    """Retrieve all text from a PDF file.

    Arguments:
        pdf Path of the file to read.
        page: Number of the page to read. If None, read all the pages.

    Returns:
        A list of lines of text.
    """
    if page is None:
        args = ['pdftotext', '-layout', '-q', pdf, '-']
    else:
        args = ['pdftotext', '-f', str(page), '-l', str(page), '-layout',
                '-q', pdf, '-']
    try:
        txt = subprocess.check_output(args, universal_newlines=True)
        lines = txt.splitlines()
    except subprocess.CalledProcessError:
        lines = []
    return [normalizar(line.lower()) for line in lines]


lineas = pdftotext('ciencias_horarios/20211.pdf')

Este segmento de código puede ser consultado en https://stackoverflow.com/questions/52683133/text-scraping-a-pdf-with-python-pdfquery

Veamos cómo se ven nuestra variable:

In [31]:
#Quitamos primeras lineas

lineas = lineas[7:]
if __name__=='__main__':
    lineas[:20]

A continuación, también usaremos como referencia el documento `materias`

In [32]:
mat = open('ciencias_horarios/materias','r')

reemplazo=(
    ('\t10',''),
    ('\t12',''),
    ('\t18',''),
    ('\t',' '),
    ('\n',''))

materias = mat.readlines()

for idx in range(len(materias)):
    materia = materias[idx]
    for a,b in reemplazo:
        materia = materia.replace(a,b)
    materias[idx] = normalizar(materia.lower())

mat.close()

Y para la carrera de Matemáticas, usaremos el plan de estudios de 1983, dividiendo las materias por semestre y optativas:

In [33]:
def get_materia_clave(materias):
    '''
    Función que devolverá un diccionario con las claves y nombre de las materias
    '''
    dic_materias = {}
    for materia in materias:
        clave = materia[:4]
        nom_materia = materia[5:]
        dic_materias[clave] = nom_materia
    return dic_materias

primer_sem = get_materia_clave(materias[3:7])
segundo_sem = get_materia_clave(materias[12:15])
tercer_sem = get_materia_clave(materias[20:22])
cuarto_sem = get_materia_clave(materias[27:30])
quinto_sem = get_materia_clave(materias[35:38])
sexto_sem = get_materia_clave(materias[43:44])
optativas_1 = get_materia_clave(materias[49:66])
optativas_2 = get_materia_clave(materias[71:106])
optativas_3 = get_materia_clave(materias[111:])

lista_materias = [primer_sem,
                 segundo_sem,
                 tercer_sem,
                 cuarto_sem,
                 quinto_sem,
                 sexto_sem,
                 optativas_1,
                 optativas_2,
                 optativas_3]

In [138]:
with open('ciencias_horarios/semestres/1.json','w') as archivo:
    json.dump(primer_sem, archivo)
with open('ciencias_horarios/semestres/2.json','w') as archivo:
    json.dump(segundo_sem, archivo)
with open('ciencias_horarios/semestres/3.json','w') as archivo:
    json.dump(tercer_sem, archivo)
with open('ciencias_horarios/semestres/4.json','w') as archivo:
    json.dump(cuarto_sem, archivo)
with open('ciencias_horarios/semestres/5.json','w') as archivo:
    json.dump(quinto_sem, archivo)
with open('ciencias_horarios/semestres/6.json','w') as archivo:
    json.dump(sexto_sem, archivo)
with open('ciencias_horarios/semestres/op1.json','w') as archivo:
    json.dump(optativas_1, archivo)
with open('ciencias_horarios/semestres/op2.json','w') as archivo:
    json.dump(optativas_2, archivo)
with open('ciencias_horarios/semestres/op3.json','w') as archivo:
    json.dump(optativas_3, archivo)

In [102]:
optativas_1

{'0014': 'algebra geometrica',
 '0088': 'conjuntos convexos',
 '0271': 'conjuntos y logica',
 '0574': 'diseño de sistemas digitales',
 '0419': 'electromagnetismo i',
 '0302': 'fenomenos colectivos',
 '0250': 'geometria moderna ii',
 '0251': 'geometria proyectiva',
 '0422': 'graficas y juegos',
 '0224': 'introduccion a ciencias de la computacion i',
 '0339': 'introduccion a ciencias de la computacion ii',
 '0272': 'introduccion a la geometria avanzada',
 '0416': 'matematicas discretas',
 '0228': 'mecanica vectorial',
 '0625': 'probabilidad i',
 '0764': 'teoria de los numeros i',
 '0777': 'teoria de los numeros ii'}

A continuación usaremos la biblioteca de expresiones regulares y las funciones de análisis de patrones:

In [35]:

prueba = lineas[10]

def is_inicio_materia(linea):
    '''
    Dada una cadena de texto, identifica si es indicadora de materia,
    es decir, si contiene la clave, nombre y grupo de una materia
    
    :param linea: Cadena de texto a analizar
    
    :returns Bool: Indicadora sobre si la cadena contiene información
                   acerca de una materia.
    '''
    patron_inicio_materia = '^\s*\d{4}\s*\d*\s*'
    materia = re.match(patron_inicio_materia,linea)
    if materia is None:
        return False
    return True

def informacion_materia(texto):
    '''
    Dado un texto, identifica las tuplas con el inicio y fin de aquellas líneas de texto que delimitan 
    la información de una materia como profesor, nombre, horarios.
    Ej:
    Dada la cadena
    ['        ayudante    edgar ladxidua saynes rueda',
 '     0007     10    algebra superior i                      4001      47',
 '         profesor   francisco marmolejo rivas                        lu mi vi   9 a 10',
 '        ayudante    bedani fernanda mendez muciño                    ma ju    9 a 10',
 '        ayudante    tonatiuh matos wiederhold',
 '     0007     10    algebra superior i                      4002      57',
 '         profesor   adriana leon montes                             lu mi vi   9 a 10',
 '        ayudante    luis arturo acosta aldaz                           ma ju    9 a 10',
 '        ayudante    arturo lopez gonzalez',
 '     0007     10    algebra superior i                      4003      41',
 '         profesor   ernesto mayorga saucedo                          lu mi vi   9 a 10',
 '        ayudante    luis alberto jimenez ramirez                     ma ju    9 a 10',
 '        ayudante    octavio daniel rios garcia',]
     Regresará
     [(1, 5), (5, 9), (9, 9)]
     Pues son lineas que contienen información de materias, como no hay otra materia que empieza después de la última,
     esta solo detectará el inicio y no el fin, pues detecta el fin de una materia si empieza otra o hay un salto de página
     
     :param [str] texto: Lista de cadenas a analizar
     
     :returns: Tuplas ordenadas según las líneas que contienen información sobre la materia.
    '''
    # Una materia acaba cuando empieza otra o hay una cadena de texto vacía
    patron_inicio_materia = '^\s*\d{4}\s*\d*\s*'
    n = len(texto)
    materias_=[]
    i = 0
    while i < n:
        linea = texto[i]
        if is_inicio_materia(linea):
            inicio = i
            for ii in range(i+1,n):
                linea = texto[ii]
                if linea=='' or is_inicio_materia(linea):
                    fin = ii
                    break
            i = ii
            materias_.append((inicio,fin))
        else:
            i += 1
    return materias_

El siguiente paso es dividir el texto de acuerdo a las materias:

In [36]:
materias_ = []
for inicio,fin in informacion_materia(lineas):
    materias_.append(lineas[inicio:fin])

info_optativas = lineas[2085:]
for inicio,fin in informacion_materia(info_optativas):
    materias_.append(info_optativas[inicio:fin])


Ahora definimos funciones para la obtención de datos de cada materia

In [37]:
def get_clave_grupo(materia):
    '''
    Dentro de una cadena con la información de una materia, recupera
    la clave de materia y su grupo
    
    :param str materia: Cadena de texto de la cual se recuperará la información.
    
    :returns clave,grupo: Clave y grupo de la materia
    '''
    patron_inicio = '\s*\d{4}'
    coincidencias = re.finditer(patron_inicio, materia)
    clave_idx = next(coincidencias).end()-4
    grupo_idx = next(coincidencias).end()-4
    clave = materia[clave_idx:clave_idx+4]
    grupo = materia[grupo_idx:grupo_idx+4]
    
    return clave, grupo

def get_profesor(materia):
    '''
    Dada la información de una materia, recupera el 
    nombre del profesor
    :param [str] materia: Lista de cadenas de texto de la cuál se recuperará el nombre del profesor.
    
    :returns str: Nombre del profesor.
    '''
    patron_profesor = '\s*profesor\s*'
    for linea in materia:
        profesor_idx = re.search(patron_profesor,linea)
        if profesor_idx is not None:
            #print(linea)
            patron_fin_profesor = '\s{10}'
            profesor_end = re.search(patron_fin_profesor,linea).start()
            profesor = linea[profesor_idx.end():profesor_end]
            return profesor
    return None

def get_hora(linea):
    '''
    Regresa la hora de clase indicada en la linea si es que hay.
    
    :param str linea: Dada una cadena de texto, analiza si contiene un horario
                     y recupera la hora sin importar qué días se imparte
    
    :returns None: En caso de que la línea no contenga hora.
    :returns [inicio flt,fin flt]: Hora de inicio y término de la materia.
    '''
    patron_hora = '\d+\s*a\s*\d+'
    hora_match = re.search(patron_hora,linea)
    if hora_match is not None:
        hora_match = hora_match.start()
        hora_str = linea[hora_match:].replace(':30','.5')
        separador = hora_str.find('a')
        inicio = hora_str[:separador]
        fin = hora_str[separador+1:]
        return [float(inicio),float(fin)]
    return None
    
def get_dia_hora(linea):
    '''
    Dada una linea, indica los días de la semana en la que se tiene
    clase junto a sus horas
    
    :param str linea: Cadena de texto a analizar
    
    :returns dict dias: Diccionario con los días de la semana como llaves y sus horarios.
    '''
    dias = {
        'lu':[],
        'ma':[],
        'mi':[],
        'ju':[],
        'vi':[],
        'sa':[]
    }
    
    for dia in dias.keys():
        if dia=='lu':
             if 'lu a vi' in linea:
                dias['lu'].append(get_hora(linea))
                dias['ma'].append(get_hora(linea))
                dias['mi'].append(get_hora(linea))
                dias['ju'].append(get_hora(linea))
                dias['vi'].append(get_hora(linea))
                break
            
             elif 'lu a sa' in linea:
                dias['lu'].append(get_hora(linea))
                dias['ma'].append(get_hora(linea))
                dias['mi'].append(get_hora(linea))
                dias['ju'].append(get_hora(linea))
                dias['vi'].append(get_hora(linea))
                dias['sa'].append(get_hora(linea))
                break
                
             elif ' '+dia+' ' in linea:
                dias[dia].append(get_hora(linea))
        elif ' '+dia+' ' in linea:
            dias[dia].append(get_hora(linea))
    
    return dias

Ahora, procedemos a definir funciones que combinan las horas de las materias y finalmente consiguen el horario de una materia.

In [38]:
def mergeDict(dict1, dict2):
    
    ''' Combina los días de distintos diccionarios y combina horas
    consecutivas.
    
    :param dict dict1: Diccionario a combinar con días de la semana y horas
    :param dict dict2: Diccionario a combinar con días de la semana y horas
    
    :returns dict dict3: Diccionarios con la información acerca de un horario combinando horas consecutivas.
    
    '''
    dict3 = {**dict1, **dict2}
    for key, value in dict3.items():
        if key in dict1 and key in dict2:
            dict3[key] =  dict1[key] + value
            
        if len(dict3[key])>1:
            if dict3[key][0][1] == dict3[key][1][0]:
                dict3[key] = [[dict3[key][0][0],dict3[key][1][1]]]
            elif dict3[key][0][0] == dict3[key][1][1]:
                dict3[key] = [[dict3[key][1][0],dict3[key][0][1]]]
        
    return dict3

def get_horario(materia):
    '''
    Dada la información de una materia, recupera el diccionario
    con sus días y sus horas.
    
    :param [str] materias: Lista con cadenas sobre la información de una materia para obtener horario
    
    :returns dict: Diccionario con los días y horas en las que se imparte dicha materia.
    '''
    horario = {}
    for linea in materia:
        horario = mergeDict(get_dia_hora(linea), horario)
    return horario



Finalmente, obtenemos la información de todas las materias con ayuda de la siguiente función:

In [39]:
def get_datos_materia(materia):
    '''
    Dada una materia, obtiene su clave, grupo, profesor y horario
    
    :param [str] materia: Lista de cadenas de texto de las qué recuperar datos.
    
    :returns dict: Datos de las materias con las llaves siendo el campo de información y los
                    valores, los datos.
    '''
    clave, grupo = get_clave_grupo(materia[0])
    for semestre in lista_materias:
        if clave in semestre.keys():
            materia_nom = semestre[clave]
            break
    
    try:
        profesor = get_profesor(materia)
    except Exception as e:
        profesor = 'Profesor Asignatura'
    horario = get_horario(materia)
    datos = {
        'clave':clave,
        'materia':materia_nom,
        'grupo':grupo,
        'profesor':profesor,
        'horario':horario
    }
    return datos

materias_limpio = []
for materia in materias_:
    materias_limpio.append(get_datos_materia(materia))


In [104]:

materias_limpio[:2]

[{'clave': '0007',
  'materia': 'algebra superior i',
  'grupo': '4000',
  'profesor': 'eugenia marmolejo rivas',
  'horario': {'lu': [[9.0, 10.0]],
   'ma': [[9.0, 10.0]],
   'mi': [[9.0, 10.0]],
   'ju': [[9.0, 10.0]],
   'vi': [[9.0, 10.0]],
   'sa': []}},
 {'clave': '0007',
  'materia': 'algebra superior i',
  'grupo': '4001',
  'profesor': 'francisco marmolejo rivas',
  'horario': {'lu': [[9.0, 10.0]],
   'ma': [[9.0, 10.0]],
   'mi': [[9.0, 10.0]],
   'ju': [[9.0, 10.0]],
   'vi': [[9.0, 10.0]],
   'sa': []}}]

Guardamos el diccionario obtenido en un archivo:        

In [135]:
with open('ciencias_horarios/materias_limpio.json','w') as archivo:
    json.dump(materias_limpio, archivo)